<h1 style="text-align: center">Customising nbconvert</h1>

<h3 style="text-align: center">How to turn Jupyter notebooks into anything you want</h3>

---

<p style="font-weight:bold; text-align: center">Thomas Kluyver & Min Ragan-Kelley</p>

<p style="text-align: center">PyData London 2016</p>

<p style="text-align: center; font-style: italic;">Github: takluyver/customising-nbconvert</p>

<style>
.reveal pre code {
    padding: 5px;
    background-color: #F0F0F0;
    color: #1C2B9E;
}

.reveal code {
    color: #047152;
}
</style>

## Jupyter and IPython

**Jupyter** is the language agnostic parts:

- Kernel messaging protocol
- Notebook format & web interface
- JupyterHub, Nbviewer, Nbgrader...

**IPython** is interactive tools for Python:

 - Python kernel (execution, introspection, completion)
 - `%magic`,  `!system`
 - `ipython` terminal shell

## Introducing nbconvert

![nbconvert flow chart](nbconvert.png)

## Nbconvert at the command line

With files:

    jupyter nbconvert --to slides Presentation.ipynb
    # This creates Presentation.html

With pipes:

    ... | jupyter nbconvert --stdin --execute --to notebook --stdout | ...

## Aside: downgrading notebooks

If your advisor refuses to upgrade:

    jupyter nbconvert --to notebook --nbformat 3 Presentation.ipynb

Saves `Presentation.v3.ipynb`

## Nbconvert as a Python API

In [22]:
import nbformat
from nbconvert.exporters import RSTExporter

nb = nbformat.read('Presentation.ipynb', as_version=4)
rst, resources = RSTExporter().from_notebook_node(nb)
slide_start = rst.find('Nbconvert as a Python API')
print(rst[slide_start:slide_start+200])

Nbconvert as a Python API
-------------------------

.. code:: python

    import nbformat
    from nbconvert.exporters import MarkdownExporter
    
    nb = nbformat.read('Presentation.ipynb', as_ver


Nbparameterise [sample notebook](http://nbviewer.jupyter.org/github/takluyver/nbparameterise/blob/master/examples/Stock%20display.ipynb) and [demo](http://localhost:3131/)

## Nbconvert as a web service

In [21]:
from IPython.display import IFrame
IFrame('http://nbviewer.jupyter.org/url/norvig.com/ipython/Countdown.ipynb', width=800, height=600)

## Templates

In [44]:
from IPython.display import IFrame
IFrame(src='template_structure.html', width=800, height=600)

## Custom templates

    {%- extends 'full.tpl' -%}
    
    {%- block markdowncell -%}
      <div style="background-color: hsl(85, 86%, 76%);">
        {# super() means 'put here whatever the parent template does for this block' #}
        {{ super() }}
      </div>
    {%- endblock markdowncell -%}

Demos:

- Make it pop: [template](https://github.com/takluyver/customising-nbconvert/blob/master/templates/makeitpop.tpl), [result](templates/makeitpop.html)
- Foldable cells: [template](https://github.com/takluyver/customising-nbconvert/blob/master/templates/foldcode.tpl), [result](templates/foldcode.html)

## Templates using metadata

`cell` is accessible as a variable inside relevant blocks.

In a Latex template:
    
    ((* block input_group *))
      ((*- if not cell.metadata.get('nbconvert', {}).get('hide_code', False) -*))
        ((( super() )))
      ((*- endif -*))
    ((* endblock input_group *))

Exporting to slides uses metadata.

## Custom exporters

Exporters are more powerful than templates

* `NotebookExporter` uses nbformat
* `PDFExporter` extends `LatexExporter`: fills template, runs Latex
* Another PDF exporter could use Reportlab, or create HTML and 'print' to PDF

Inherit from `nbconvert.Exporter` or `nbconvert.TemplateExporter`:

In [40]:
from nbconvert import Exporter

class NewFormatExporter(Exporter):
    output_mimetype = 'application/newformat'

    def _file_extension_default(self):
        return '.nf'

    def from_notebook_node(self, nb, resources=None):
        # nb is a notebook object: use nb.metadata, nb.cells[0], etc.
        # resources, if passed, is a dictionary with extra information
        ...
        return output, resources

## Shipping custom exporters

Import path on the command line:

    nbconvert --to tpllib.MyExporter ...

-----
Or use entry points:

    [nbconvert.exporters]
    detail = mymodule:DetailExporter

Then:

    nbconvert --to detail ...

<h1 style="text-align: center">Questions?</h1>


---

<p style="text-align: center; font-style: italic;">Github: takluyver/customising-nbconvert</p>